# Reaction Property Packages in IDAES

<div class="alert alert-block alert-info">
<b>Note:</b>
Reaction property packages are closely related to, and dependent on, thermophysical property packages and it is advised the readers start with understanding these.
</div>

Similar to thermophysical property packages, reaction property packages in IDAES are used to define the set of parameters, variables and constraints associated with a specific set of chemical reactions that a user wishes to model. One of the features of the IDAES Integrated Platform is the ability for modelers to create their own property “packages” to calculate these properties, allowing them to customize the level of complexity and rigor to suit each application. This tutorial will introduce you to the basics of creating property packages for calculating reaction properties within the IDAES Core Modeling Framework.

## Relationship with Thermophysical Property Packages

Reaction properties depend on the state of the system, such as the temperature, pressure and composition of the material. All of these properties are defined in the thermophysical property package, thus reaction property packages are closely tied to thermophysical property packages; indeed, a given reaction package is often tied to a single specific thermophysical property package. Reaction packages need to be used with a thermophysical property package which defines the expected set of components and the expected forms and units for the state variables.

As such, developers of reaction packages should have a specific thermophysical property package in mind when developing a reaction property package, and to tailor the reaction package to the thermophysical property package.

## Types of Reactions

Within the IDAES Core Modeling Framework, chemical reactions are divided into two categories”

1. Equilibrium based reactions, where extent of reaction is determined by satisfying a constraint relating the concentration of species within the system, and
2. Rate based reactions, where extent of reaction depends on some characteristic of the reactor unit. Despite the name, this category is also used to represent stoichiometric and yield based reactions.

## Steps in Creating a Reaction Property Package

Creating a new property package can be broken down into the following steps, which will be demonstrated in the next part of this tutorial.

1. Defining the **units of measurement** for the property package.
2. Defining the **properties supported** by the property package and the associated metadata.
3. Defining the **equilibrium reactions** of interest.
4. Defining the **equilibrium reactions** of interest.
5. Defining the **parameters** related to the reactions of interest.
6. Creating **variables and constraints** to describe the reactions of interest.
7. Creating an **initialization routine** for the reaction property package.
8. Defining **interface methods** used to couple the property package with unit models.

# Tutorial Example

For this tutorial, we will be building a upon the property package from the thermophysical property example. In that example, we constructed a thermophysical property package that could be used to model a process for the hydrodealkylation of toluene to form benzene. This process involves five key chemical species:

* toluene
* benzene
* hydrogen
* methane
* diphenyl

In this tutorial, we will write a reaction property package to define the reactions associated with the HDA process:

$$
\text{Toluene} + \text{Hydrogen} \rightarrow \text{Benzene} + \text{Methane}
$$
$$
2 \text{Benzene} \rightleftharpoons \text{Hydrogen} + \text{Diphenyl}
$$

## A Note on this Tutorial

The `build` methods in the reaction property package classes are generally written as a single, long method. However, to break the code in the manageable pieces for discussion, in this tutorial we will create a number of smaller sub-methods that will then be called as part of the `build` method. This is done entirely for presentation purposes, and model developers should not feel compelled to write their models this way.
    
An example of how the example in this tutorial would be written without sub-methods can be found in the same folder with the name [`reaction_property_example.py`](files/reaction_property_example.py).

## Components of a Reaction Property Package

Similar to thermophysical property packages, reaction property packages consist of three parts, which are written as Python `classes`. These components are:

* The `Reaction Parameter Block` class, which contains all the global parameters associated with the reaction property package,
* The `Reaction Block Data` class, which contains the instructions on how to calculate all the properties at a given state, and,
* The `Reaction Block` class, which is used to construct indexed sets of `Reaction Block Data` objects and contains methods for acting on all multiple `Reaction Block Data` objects at once (such as initialization).

It is not necessary to understand the reason for the distinction between the `Reaction Block` and `Reaction Block Data` classes. Suffice to say that this is due to the need to replicate the underlying component structure of Pyomo, and that the `Reaction Block` represents the indexed `Block` representing a set of states across a given indexing set (most often time), and the `Reaction Block Data` represents the individual elements of the indexed `Block`.

## Importing Libraries

Before we begin writing the actual `classes` however, we need to import all the necessary components from the Pyomo and IDAES modeling libraries. To begin with, we are going to need a number of components from the Pyomo modeling environment to construct the variables, constraints and parameters that will make up the property package, and we will also make use of the Pyomo units of measurement tools to define the units of our properties. We will also make use of a number of components and supporting methods from the IDAES modeling framework and libraries.

Rather than describe the purpose of all of these here, we shall just import all of them here and discuss their use as they arise in the example.

In [ ]:
# Import Pyomo libraries
from pyomo.environ import (Constraint,
                           exp,
                           Param,
                           Set,
                           units as pyunits,
                           Var)

# Import IDAES cores
from idaes.core import (declare_process_block_class,
                        MaterialFlowBasis,
                        ReactionParameterBlock,
                        ReactionBlockDataBase,
                        ReactionBlockBase)
from idaes.core.util.constants import Constants as const
import idaes.logger as idaeslog

# The Reaction Parameter Block

We will begin by constructing the `Reaction Parameter Block` for our example. This serves as the central point of reference for all aspects of the reaction property package, and needs to define a number of things about the package. These are summarized below:

* Units of measurement
* What reaction properties are supported and how they are implemented
* All the global parameters necessary for calculating properties
* A reference to the associated `Reaction Block` class, so that construction of the `Reaction Block` components can be automated from the `Reaction Parameter Block`

## Step 1: Define Units of Measurement and Property Metadata

The first step is to define the units of measurement for the property package, which will in turn be inherited by any unit model using this property package. The IDAES Core Modeling Framework requires that the units of measurement defined for a reaction property package be identical to those used in the thermophysical property package it is associated with (this is to avoid any chance of confusion regarding units when setting up the balance equations).

In order to set the base units, we use the same approach as for thermophysical property packages; we create a dictionary which has each of the base quantities as a key, and provide a Pyomo recognized unit as the value as shown in the cell below.

Much like thermophysical property packages, we also need to define metadata regarding the reaction properties supported by our package. For this example, we have three supported properties:

* a rate constant (`k_rxn`),
* an equilibrium constant (`k_eq`), and
* a reaction rate term (`rate_reaction`).

The cell below shows how to define the units of measurement and properties metadata for this example.

In [ ]:
units_metadata = {'time': pyunits.s,
                  'length': pyunits.m,
                  'mass': pyunits.kg,
                  'amount': pyunits.mol,
                  'temperature': pyunits.K}

properties_metadata = {'k_rxn': {'method': None},
                       'k_eq': {'method': None},
                       'reaction_rate': {'method': None}}

In [ ]:
def define_kinetic_reactions(self):
    # Rate Reaction Index
    self.rate_reaction_idx = Set(initialize=["R1"])

    # Rate Reaction Stoichiometry
    self.rate_reaction_stoichiometry = {("R1", "Vap", "benzene"): 1,
                                        ("R1", "Vap", "toluene"): -1,
                                        ("R1", "Vap", "hydrogen"): -1,
                                        ("R1", "Vap", "methane"): 1,
                                        ("R1", "Vap", "diphenyl"): 0}

In [ ]:
def define_equilibrium_reactions(self):
    # Equilibrium Reaction Index
    self.equilibrium_reaction_idx = Set(initialize=["E1"])

    # Equilibrium Reaction Stoichiometry
    self.equilibrium_reaction_stoichiometry = {
        ("E1", "Vap", "benzene"): -2,
        ("E1", "Vap", "toluene"): 0,
        ("E1", "Vap", "hydrogen"): 1,
        ("E1", "Vap", "methane"): 0,
        ("E1", "Vap", "diphenyl"): 1}

In [ ]:
def define_parameters(self):
    # Arrhenius Constant
    self.arrhenius = Param(default=1.25e-9,
                           doc="Arrhenius constant",
                           units=pyunits.mol/pyunits.m**3/pyunits.s/pyunits.Pa**2)

    # Activation Energy
    self.energy_activation = Param(default=3800,
                                   doc="Activation energy",
                                   units=pyunits.J/pyunits.mol)

In [ ]:
@declare_process_block_class("HDAReactionParameterBlock")
class HDAReactionParameterData(ReactionParameterBlock):
    """
    Reaction Parameter Block Class
    """

    def build(self):
        '''
        Callable method for Block construction.
        '''
        super(HDAReactionParameterData, self).build()

        self._reaction_block_class = HDAReactionBlock
        
        define_kinetic_reactions(self)
        define_equilibrium_reactions(self)
        define_parameters(self)

    @classmethod
    def define_metadata(cls, obj):
        obj.add_properties(properties_metadata)
        obj.add_default_units(units_metadata)

## Reaction Blocks

Even more text

In [ ]:
class _HDAReactionBlock(ReactionBlockBase):

    def initialize(blk, outlvl=idaeslog.NOTSET, **kwargs):
        init_log = idaeslog.getInitLogger(blk.name, outlvl, tag="properties")
        init_log.info('Initialization Complete.')

In [ ]:
def define_variables_and_parameters(self):
    self.k_rxn = Var(initialize=7e-10,
                    doc="Rate constant",
                    units=pyunits.mol/pyunits.m**3/pyunits.s/pyunits.Pa**2)

    self.k_eq = Param(initialize=10000,
                      doc="Equlibrium constant",
                      units=pyunits.Pa)

    self.reaction_rate = Var(self.params.rate_reaction_idx,
                             initialize=0,
                             doc="Rate of reaction",
                             units=pyunits.mol/pyunits.m**3/pyunits.s)

In [ ]:
def define_rate_expression(self):
    self.arrhenius_equation = Constraint(
            expr=self.k_rxn == self.params.arrhenius * exp(
                -self.params.energy_activation /
                (const.gas_constant*self.state_ref.temperature)))

    def rate_rule(b, r):
        return b.reaction_rate[r] == (
                    b.k_rxn *
                    b.state_ref.mole_frac_comp["toluene"] *
                    b.state_ref.mole_frac_comp["hydrogen"] *
                    b.state_ref.pressure**2)
    self.rate_expression = Constraint(self.params.rate_reaction_idx,
                                      rule=rate_rule)

In [ ]:
def define_equilibrium_expression(self):
    self.equilibrium_constraint = Constraint(
        expr=self.k_eq *
        self.state_ref.mole_frac_comp["benzene"] *
        self.state_ref.pressure ==
        self.state_ref.mole_frac_comp["diphenyl"] *
        self.state_ref.mole_frac_comp["hydrogen"] *
        self.state_ref.pressure**2)

In [ ]:
@declare_process_block_class("HDAReactionBlock",
                             block_class=_HDAReactionBlock)
class HDAReactionBlockData(ReactionBlockDataBase):
    def build(self):

        super(HDAReactionBlockData, self).build()
        
        define_variables_and_parameters(self)
        define_rate_expression(self)
        define_equilibrium_expression(self)
    
    def get_reaction_rate_basis(b):
        return MaterialFlowBasis.molar

# Demonstration

In [ ]:
from pyomo.environ import ConcreteModel
from pyomo.util.check_units import assert_units_consistent

from idaes.core import FlowsheetBlock
from idaes.core.util import get_solver
from idaes.generic_models.unit_models import CSTR

from thermophysical_property_example import HDAParameterBlock

from idaes.core.util.model_statistics import degrees_of_freedom

In [ ]:
m = ConcreteModel()

m.fs = FlowsheetBlock(default={"dynamic": False})

m.fs.thermo_params = HDAParameterBlock()
m.fs.reaction_params = HDAReactionParameterBlock(
    default={"property_package": m.fs.thermo_params})

m.fs.reactor = CSTR(default={
    "property_package": m.fs.thermo_params,
    "reaction_package": m.fs.reaction_params,
    "has_equilibrium_reactions": True})

In [ ]:
print("Degrees of Freedom: ", degrees_of_freedom(m))

In [ ]:
m.fs.reactor.inlet.flow_mol.fix(100)
m.fs.reactor.inlet.temperature.fix(500)
m.fs.reactor.inlet.pressure.fix(350000)
m.fs.reactor.inlet.mole_frac_comp[0, "benzene"].fix(0.1)
m.fs.reactor.inlet.mole_frac_comp[0, "toluene"].fix(0.4)
m.fs.reactor.inlet.mole_frac_comp[0, "hydrogen"].fix(0.4)
m.fs.reactor.inlet.mole_frac_comp[0, "methane"].fix(0.1)
m.fs.reactor.inlet.mole_frac_comp[0, "diphenyl"].fix(0.0)

m.fs.reactor.volume.fix(1)

print("Degrees of Freedom: ", degrees_of_freedom(m))

In [ ]:
m.fs.reactor.initialize(state_args={
    "flow_mol": 100,
    "mole_frac_comp": {
        "benzene": 0.15,
        "toluene": 0.35,
        "hydrogen": 0.35,
        "methane": 0.15,
        "diphenyl": 0.01},
    "temperature": 600,
    "pressure": 350000})

solver = get_solver()
results = solver.solve(m, tee=True)

In [ ]:
from pyomo.environ import TerminationCondition, SolverStatus
assert results.solver.termination_condition == TerminationCondition.optimal
assert results.solver.status == SolverStatus.ok

In [ ]:
m.fs.reactor.outlet.display()

In [ ]:
import pytest
from pyomo.environ import value
assert value(m.fs.reactor.outlet.flow_mol[0]) == pytest.approx(100, abs=1e-3)
assert value(m.fs.reactor.outlet.temperature[0]) == pytest.approx(790.212, abs=1e-3)
assert value(m.fs.reactor.outlet.mole_frac_comp[0, "benzene"]) == pytest.approx(0.159626, abs=1e-6)

In [ ]:
assert_units_consistent(m)

# Using the Generic Property Framework

Show how to do the same thing with the generic framework.
